## Import libs

In [118]:
import pandas as pd
import numpy as np
import sqlite3
from tqdm.notebook import tqdm

In [165]:
users = pd.read_csv('../Board-Games/data/full/bgg_users_full.csv')
users

,link,nickname,country,state_cuty
0,/user/ me,me,England,England</div>
1,/user/00296,00296,England,England<br/>US Armed Forces - Europe</div>
2,/user/08bhattrell,08bhattrell,England,England</div>
3,/user/08pm,08pm,England,England<br/>Michigan</div>
4,/user/100pcBlade,100pcBlade,England,England<br/>Crawley<br/>West Sussex</div>
...,...,...,...,...
1665199,/user/UndergroundMan,UndergroundMan,Tanzania,Tanzania<br/>Unknown Village<br/>Arusha Region...
1665200,/user/vortexturd,vortexturd,Tanzania,Tanzania</div>
1665201,/user/woodwardmw,woodwardmw,Tanzania,Tanzania<br/>Dar es Salaam<br/>Dar es Salaam</...
1665202,/user/X3N0,X3N0,Tanzania,Tanzania<br/>Unknown Village<br/>Arusha Region...


In [3]:
ratings = pd.read_feather('../Board-Games/data/full/bgg_ratings_full.feather')
ratings

,nickname,title,boardgame_id,rating,num_of_plays,comment,own,prevowned,fortrade,want,wanttoplay,wanttobuy,wishlist,preordered,last_modified
0,DiVo1975,Anachrony,185343,9.0,2,None,0,1,0,0,0,0,0,0,2022-11-06 14:02:30
1,DiVo1975,Anachrony: Infinity Box,278292,10.0,2,None,1,0,0,0,0,0,0,0,2022-01-06 06:21:59
2,DiVo1975,Ark Nova,342942,10.0,25,None,1,0,0,0,0,0,0,0,2022-09-05 01:21:16
3,DiVo1975,Arkham Horror: The Card Game,205637,9.0,0,None,1,0,0,0,0,0,0,0,2022-11-06 14:02:55
4,DiVo1975,Blood Rage,170216,10.0,1,None,1,0,0,0,0,0,0,0,2022-11-06 14:03:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20186643,zinnwen,Taj Mahal,475,7.0,0,None,0,0,0,0,0,0,0,0,2009-05-25 03:54:26
20186644,zinnwen,Through the Ages: A New Story of Civilization,182028,9.0,0,None,0,0,0,0,0,0,0,0,2017-09-01 11:38:04
20186645,zinnwen,Ticket to Ride,9209,6.0,0,None,0,0,0,0,0,0,0,0,2009-05-25 03:54:31
20186646,zinnwen,Tigris & Euphrates,42,8.0,0,None,0,0,0,0,0,0,0,0,2009-05-25 03:53:41


In [4]:
boardgames = pd.read_feather('../Board-Game-Scraper/bgg_boardgames.feather')
boardgames

,boardgame_id,title,year_published,minplayers,maxplayers,minplaytime,maxplaytime,age,users_rated,average_rating,...,graphic_designers,subdomains,implementations,suggested_numplayers,podcast_episodes,comments,marketplace_history,marketplace_listings,thumbnail_link,image_link
0,5848,Rockit to Number 1,2002,2,8,40,40,0,10,5.825,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,King4alifetime|Would love to see an update on ...,,,https://cf.geekdo-images.com/pNnHBeoBxmZ2GajhG...,https://cf.geekdo-images.com/pNnHBeoBxmZ2GajhG...
1,259048,Star Wars: X-Wing (Second Edition) – RZ-2 A-Wi...,2018,2,2,30,45,14,39,8.0865,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,cammo35|For Outter Rim||Cullensdad|Own 4 copie...,,,https://cf.geekdo-images.com/N0aX_CzHdNSzBEREy...,https://cf.geekdo-images.com/N0aX_CzHdNSzBEREy...
2,122324,Wer bin ich? Das Promi-Rate-Spiel,2003,2,6,40,40,12,0,0,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,https://cf.geekdo-images.com/7Lc3uvftR0sI9TGtj...,https://cf.geekdo-images.com/7Lc3uvftR0sI9TGtj...
3,239964,Slam Burger,2017,4,8,0,0,6,7,6.92871,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,bert8freudenkinder|fun!||fs1973|LeiriaCON copy,unix_saledate|1521510090||saletime|85885||pric...,,https://cf.geekdo-images.com/wIdJWjYScBDpUbspX...,https://cf.geekdo-images.com/wIdJWjYScBDpUbspX...
4,227582,Cthulhu: Rise of the Cults,2017,1,4,90,120,14,109,6.57199,...,,,,1||Best~0|Recommended~0|Not Recommended~1|||2|...,,adamdynris|Kickstarter janvier 2018||AgentQ|I ...,unix_saledate|1516883073||saletime|4553993||pr...,,https://cf.geekdo-images.com/9ORSJup7z0llS8zQN...,https://cf.geekdo-images.com/9ORSJup7z0llS8zQN...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134735,373860,Tricky Rails,2022,4,4,45,60,12,5,8.2,...,,,,4+||,,BankofDracula|DJ Kenel has been mucking around...,,,https://cf.geekdo-images.com/SVt_sNGFFOI40hCq2...,https://cf.geekdo-images.com/SVt_sNGFFOI40hCq2...
134736,114857,Dougram Simulation Manual,1982,2,2,30,30,0,1,6,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,AMI_SOLGER|Fast action tactical combat game.\n...,,,https://cf.geekdo-images.com/bjdWaEjC0XleaCDWF...,https://cf.geekdo-images.com/bjdWaEjC0XleaCDWF...
134737,67629,U-Build Connect 4,2010,2,2,0,0,6,33,4.18182,...,,Children's Games,,1||Best~0|Recommended~0|Not Recommended~3|||2|...,,ccmonter|JUNK BIN\n\nMost of the entertainment...,,,https://cf.geekdo-images.com/agEIfeC82HLerQVQ9...,https://cf.geekdo-images.com/agEIfeC82HLerQVQ9...
134738,304430,Boney the Dinosaur,2015,4,8,0,0,0,0,0,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,https://cf.geekdo-images.com/LVPg9GMlSFzt5csLF...,https://cf.geekdo-images.com/LVPg9GMlSFzt5csLF...


## Function for queries

In [91]:
def get_df_from_db(query: str, out=False) -> pd.DataFrame:
    if out:
        conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

    df = pd.read_sql(query, conn)

    if out:
        conn.close()
    return df

def get_data_from_db(query: str, value=None, out=False, conn=False):
    if out:
        conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')
    cursor = conn.cursor()
    
    if value:
        cursor.execute(query, value)
    else:
        cursor.execute(query)

    data = cursor.fetchall()

    if out:
        conn.close()
    return data

## Prepare users data

In [166]:
def get_user_state(x):
    try:
        return x.split('|')[1]
    except:
        return np.NaN


def get_user_city(x):
    try:
        return x.split('|')[2]
    except:
        return np.NaN

def isNaN(num):
    return num != num

users['state_cuty'] = users.state_cuty.apply(lambda x: x.replace('<br/>', '|').replace('</div>', ''))
users['city'] = users.state_cuty.apply(get_user_state)
users['state'] = users.state_cuty.apply(get_user_city)
users['link'] = users.link.apply(lambda x: 'https://boardgamegeek.com' + x)
users

,link,nickname,country,state_cuty,city,state
0,https://boardgamegeek.com/user/ me,me,England,England,NaN,NaN
1,https://boardgamegeek.com/user/00296,00296,England,England|US Armed Forces - Europe,US Armed Forces - Europe,NaN
2,https://boardgamegeek.com/user/08bhattrell,08bhattrell,England,England,NaN,NaN
3,https://boardgamegeek.com/user/08pm,08pm,England,England|Michigan,Michigan,NaN
4,https://boardgamegeek.com/user/100pcBlade,100pcBlade,England,England|Crawley|West Sussex,Crawley,West Sussex
...,...,...,...,...,...,...
1665199,https://boardgamegeek.com/user/UndergroundMan,UndergroundMan,Tanzania,Tanzania|Unknown Village|Arusha Region,Unknown Village,Arusha Region
1665200,https://boardgamegeek.com/user/vortexturd,vortexturd,Tanzania,Tanzania,NaN,NaN
1665201,https://boardgamegeek.com/user/woodwardmw,woodwardmw,Tanzania,Tanzania|Dar es Salaam|Dar es Salaam,Dar es Salaam,Dar es Salaam
1665202,https://boardgamegeek.com/user/X3N0,X3N0,Tanzania,Tanzania|Unknown Village|Arusha Region,Unknown Village,Arusha Region


In [187]:
len(users.nickname) - len(users.nickname.drop_duplicates())

0

In [183]:
users = users.drop(index=208405)
users = users.drop(index=199352)

In [186]:
users[users.nickname == 'Sau_']

,link,nickname,country,state_cuty,city,state
1543499,https://boardgamegeek.com/user/Sau_,Sau_,Sweden,Sweden|Malmö,Malmö,NaN


In [180]:
users.iloc[1543498]

link          https://boardgamegeek.com/user/Sau_
nickname                                     Sau_
country                                    Sweden
state_cuty                           Sweden|Malmö
city                                        Malmö
state                                         NaN
Name: 1543499, dtype: object

In [209]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

# execute the insert statements
for i in list(users.country.unique()):
    cursor.execute('INSERT INTO countries (country) VALUES (?)', (i,))
    
for i in list(users.state.unique()):
    if not isNaN(i):
        cursor.execute('INSERT INTO states (state) VALUES (?)', (i,))

for i in list(users.city.unique()):
    if not isNaN(i):
        cursor.execute('INSERT INTO cities (city) VALUES (?)', (i,))

        
for index, row in tqdm(users.iterrows()):
    if not isNaN(row.country):
        country = get_data_from_db(f"""
                SELECT country_id
                FROM countries 
                WHERE country = ?
                """, (row.country,), conn=conn)[0][0]
    else:
        country = np.nan

    if not isNaN(row.state):
        state = get_data_from_db(f"""
                SELECT state_id
                FROM states 
                WHERE state = ?
                """, (row.state,), conn=conn)[0][0]
    else:
        state = np.nan
        
    if not isNaN(row.city):
        city = get_data_from_db(f"""
                SELECT city_id
                FROM cities 
                WHERE city = ?
                """, (row.city,), conn=conn)[0][0]
    else:
        city = np.nan
        
    last_check = '2023-02-01'
    cursor.execute('INSERT INTO users (nickname, country_id, state_id, city_id, last_check) VALUES (?, ?, ?, ?, ?)', (row.nickname, country, state, city, last_check))
#     break
# commit the changes
conn.commit()

# close the connection
conn.close()

0it [00:00, ?it/s]

## Prepare boardgame data

In [94]:
def split_once(data: list) -> list:
    splitted = []
    for i in data:
        splitted += i.split('|')
    return list(set(splitted))


In [ ]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

# execute the insert statements
for i in list(set(list(boardgames.thumbnail_link.unique()) + list(boardgames.image_link.unique()))):
    if not isNaN(i):
        if i != '':
            cursor.execute('INSERT INTO links (link) VALUES (?)', (i,))

for i in split_once(list(set(boardgames.publishers[boardgames.publishers != '']))):
    if not isNaN(i):
            cursor.execute('INSERT INTO publishers (publisher) VALUES (?)', (i,)) 

# commit the changes
conn.commit()

# close the connection
conn.close()

In [224]:
isNaN('(Looking for a publisher)')

False

In [197]:
len(list(boardgames.title)) - len(set(boardgames.title))

4980

In [245]:
boardgames.iloc[43592]

boardgame_id                                                       140700
title                                               Perfect Storm: Alaska
year_published                                                       2016
minplayers                                                              2
maxplayers                                                              5
minplaytime                                                            90
maxplaytime                                                            90
age                                                                     8
users_rated                                                             5
average_rating                                                       5.86
bayes_average_rating                                                    0
median                                                                  0
stddev                                                            1.01902
owned                                 

In [304]:
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')
cursor = conn.cursor()
i = 'Joachim Zischke | terra authentica'

# cursor.execute('INSERT INTO publishers (publisher) VALUES (?)', (i,))
cursor.execute('''
SELECT publisher_id
FROM publishers
WHERE publisher = ?
LIMIT 10
''', (i,))


data = cursor.fetchall()
# conn.commit()
conn.close()
data

[(24080,)]

In [300]:
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')
cursor = conn.cursor()
i = 'Joachim Zischke '

# cursor.execute('INSERT INTO publishers (publisher) VALUES (?)', (i,))
cursor.execute('''
DELETE FROM publishers
WHERE publisher = ?;
''', (i,))
i = ' terra authentica'
cursor.execute('''
DELETE FROM publishers
WHERE publisher = ?;
''', (i,))


# data = cursor.fetchall()
conn.commit()
conn.close()
# data

In [297]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

# execute the insert statements

for index, row in tqdm(boardgames[boardgames.year_published != ""].iterrows()):
    try:
        if row.year_published != '0': # year_published
            year_published = int(row.year_published)
        else:
            year_published = np.nan

        if row.minplayers != '0': # min_players
            min_players = int(row.minplayers)
        else:
            min_players = np.nan

        if int(row.maxplayers) < min_players: # max_players
            max_players = np.nan
        elif row.maxplayers == '0':
            max_players = min_players
        else:
            max_players = int(row.maxplayers)

        if row.minplaytime != '0': # min_playtime
            min_playtime = int(row.minplaytime)
        else:
            min_playtime = np.nan

        if int(row.maxplaytime) < min_playtime: # max_playtime
            max_playtime = np.nan
        elif row.maxplaytime == '0':
            max_playtime = min_playtime
        else:
            max_playtime = int(row.maxplaytime)

        if (row.age != '0') & (row.age != ''): # age
            age = int(row.age)
        else:
            age = np.nan

        if row.users_rated != '': # users_rated
            users_rated = int(row.users_rated)
        else:
            users_rated = np.nan

        if row.average_rating != '': # average_rating
            average_rating = float(row.average_rating)
        else:
            average_rating = np.nan

        if row.bayes_average_rating != '': # bayes_average_rating
            bayes_average_rating = float(row.bayes_average_rating)
        else:
            bayes_average_rating = np.nan

        if row.stddev != '': # stddev
            std_dev = float(row.stddev)
        else:
            std_dev = np.nan

        if row.owned != '': # owned
            owned = int(row.owned)
        else:
            owned = np.nan 

        if row.trading != '': # trading
            trading = int(row.trading)
        else:
            trading = np.nan

        if row.wishing != '': # wishing
            wishing = int(row.wishing)
        else:
            wishing = np.nan 

        if row.num_of_comments != '': # num_of_comments
            num_of_comments = int(row.num_of_comments)
        else:
            num_of_comments = np.nan

        if row.num_of_weights != '': # num_of_weights
            num_of_weights = int(row.num_of_weights)
        else:
            num_of_weights = np.nan

        if row.average_weight != '': # average_weight
            average_weight = float(row.average_weight)
        else:
            average_weight = np.nan

        if (isNaN(row.main_publisher)) | (row.main_publisher == ''): # main_publisher_id
            main_publisher_id = np.nan
        else:
            cursor.execute('''
                            SELECT *
                            FROM publishers
                            WHERE publisher = ?
                            ''',(row.main_publisher, ))
            main_publisher_id = cursor.fetchall()[0][0]

    #         main_publisher_id = get_data_from_db(f"""
    #                 SELECT publisher_id
    #                 FROM publishers 
    #                 WHERE publisher = ?
    #                 """, (row.main_publisher,), conn=conn)[0][0]

        if (isNaN(row.thumbnail_link)) | (row.thumbnail_link == ''): # thumbnail_link_id
            thumbnail_link_id = np.nan

        else:
            cursor.execute('''
                            SELECT *
                            FROM links
                            WHERE link = ?
                            ''',(row.thumbnail_link, ))
            thumbnail_link_id = cursor.fetchall()[0][0]
    #         thumbnail_link_id = get_data_from_db(f"""
    #                 SELECT link_id
    #                 FROM links 
    #                 WHERE link = ?
    #                 """, (row.thumbnail_link,), conn=conn)[0][0]

        if (isNaN(row.image_link)) | (row.image_link == ''): # image_link_id
            image_link_id = np.nan

        else:
            cursor.execute('''
                            SELECT *
                            FROM links
                            WHERE link = ?
                            ''',(row.image_link, ))
            image_link = cursor.fetchall()[0][0]
    #         image_link_id = get_data_from_db(f"""
    #                 SELECT link_id
    #                 FROM links 
    #                 WHERE link = ?
    #                 """, (row.image_link,), conn=conn)[0][0]

    #     print((int(row.boardgame_id), row.title, year_published, min_players, max_players, 
    #            min_playtime, max_playtime, age, users_rated, average_rating, bayes_average_rating, std_dev,
    #            owned, trading, wishing, num_of_comments, 
    #            num_of_weights, average_weight, main_publisher_id, thumbnail_link_id, image_link_id))
        cursor.execute('''INSERT INTO boardgames (boardgame_id, title, year_published, min_players, max_players, 
           min_playtime, max_playtime, age, users_rated, average_rating, bayes_average_rating, std_dev,
           owned, trading, wishing, num_of_comments, num_of_weights, average_weight, main_publisher_id, 
           thumbnail_link_id, image_link_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', 
                       (int(row.boardgame_id), row.title, year_published, min_players, max_players, 
               min_playtime, max_playtime, age, users_rated, average_rating, bayes_average_rating, std_dev,
               owned, trading, wishing, num_of_comments, 
               num_of_weights, average_weight, main_publisher_id, thumbnail_link_id, image_link_id))

    except:
        print(int(row.boardgame_id))
# commit the changes
conn.commit()

# close the connection
conn.close()

0it [00:00, ?it/s]

In [296]:
boardgames[boardgames.boardgame_id.isin(['10983', '12767', '12768'])]#.main_publisher.iloc[0]

,boardgame_id,title,year_published,minplayers,maxplayers,minplaytime,maxplaytime,age,users_rated,average_rating,...,graphic_designers,subdomains,implementations,suggested_numplayers,podcast_episodes,comments,marketplace_history,marketplace_listings,thumbnail_link,image_link
43603,12768,Caraci: Das Testament,2004,2,4,30,30,0,0,0,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,,
70158,10983,Landwehr 1438,2004,2,4,45,45,11,5,5.1,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,"JasonMatthews|Beautiful, but there's some old ...",unix_saledate|1478473003||saletime|94576682||p...,,https://cf.geekdo-images.com/VLRAFPYrDkcUgMWIA...,https://cf.geekdo-images.com/VLRAFPYrDkcUgMWIA...
94543,12767,Basatiin al Djabriin,2004,2,4,75,75,0,0,0,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,https://cf.geekdo-images.com/zWCrW9UroKZBSKXzV...,https://cf.geekdo-images.com/zWCrW9UroKZBSKXzV...


In [277]:
ratings[ratings.boardgame_id == 10983] # 12768 10983 12767

,nickname,title,boardgame_id,rating,num_of_plays,comment,own,prevowned,fortrade,want,wanttoplay,wanttobuy,wishlist,preordered,last_modified
3197296,androgeus,Landwehr 1438,10983,6.0,1,None,0,1,0,0,0,0,0,0,2019-09-02 07:43:05
4646333,Banker,Landwehr 1438,10983,4.0,0,None,0,0,0,0,0,0,0,0,2010-01-09 11:28:12
7170911,JasonMatthews,Landwehr 1438,10983,2.0,1,"Beautiful, but there's some old saying about b...",0,0,0,0,0,0,0,0,2004-07-11 00:03:01
9532371,Windmilling,Landwehr 1438,10983,7.0,0,None,1,0,0,0,0,0,0,0,2018-01-05 10:24:44


In [320]:
ratings.iloc[0].comment == None

True

In [319]:
isNaN(ratings.iloc[0].comment)

False

## Prepare ratings data

In [327]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

# execute the insert statements
for i in tqdm(list(ratings.comment.unique())[1:]):
    if not isNaN(i):
#         print(i)
        cursor.execute('INSERT INTO comments (comment) VALUES (?)', (i,))

# commit the changes
conn.commit()

# close the connection
conn.close()   

  0%|          | 0/3363581 [00:00<?, ?it/s]

# STOP HERE

In [316]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

# # execute the insert statements
# for i in list(ratings.comment.unique())[1:]:
#     if not isNaN(i):
#         cursor.execute('INSERT INTO comments (comment) VALUES (?)', (i,))

for index, row in ratings.iterrows():
    cursor.execute('''
                SELECT user_id
                FROM users 
                WHERE nickname = ?
                ''',(row.nickname, ))
    user_id = cursor.fetchall()[0][0]
    
    boardgame_id = int(row.boardgame_id)
    rating = float(row.rating)
    num_of_plays = int(row.num_of_plays)
    
    if row.comment == None:
        comment_id = np.nan
    else:
        cursor.execute('''
                    SELECT comment_id
                    FROM comments 
                    WHERE comment = ?
                    ''',(row.comment, ))
        comment_id = cursor.fetchall()[0][0]
    
    own = int(row.own)
    prevowned = int(row.prevowned)
    for_trade = int(row.for_trade)
    want = int(row.want)
    want_to_play = int(row.want_to_play)
    want_to_buy = int(row.want_to_buy)
    wishlist = int(row.wishlist)
    preordered = int(row.preordered)
    last_modified = row.last_modified[:10]
    print(last_modified)
    break


# commit the changes
conn.commit()

# close the connection
conn.close()   

IndexError: list index out of range

In [364]:
ratings.nickname.unique()

array(['DiVo1975', 'happyjosiah', 'ispeakmath', ..., 'Baaz67',
       'lacarreta', 'zinnwen'], dtype=object)

In [365]:
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')
cursor = conn.cursor()
# i = 'Joachim Zischke | terra authentica'
check = []
for nick in list(ratings.nickname.unique()):
#     i = 'zombiegod'

    # cursor.execute('INSERT INTO publishers (publisher) VALUES (?)', (i,))
    cursor.execute('''
    SELECT *
    FROM users
    WHERE nickname = ?
    ''',(nick,))


    data = cursor.fetchall()
    if data == []:
        check.append(nick)
# conn.commit()
conn.close()
# data
len(check)

4488

In [352]:
ii = []
for i in data:
    ii += i
search = set(ratings.nickname) - set(ii)

In [354]:
len(search)

4488

In [53]:
get_df_from_db('''
SELECT *
FROM comments
LIMIT 10
''', out=True)

,comment_id,comment
0,1,The latest and greatest football simulation. P...
1,2,"Like it says on the label, it's a dungeon-craw..."
2,3,My favorite thing to do with a standard deck o...
3,4,A really solid addition to any game collection...
4,5,"While ostensibly a game of card drafting, 7 Wo..."
5,6,"Rated after one play, but was pretty disappoin..."
6,7,It's too silly to be an absolute favorite of m...
7,8,With a theme of corporate America and a bland-...
8,9,The theme and components work very well here. ...
9,10,This is a co-op deckbuilder. I love deckbuilde...


In [79]:
get_data_from_db(f"""
                SELECT *
                FROM users 
                """, out=True)

[(1, ' me ', 1, None, None, '2023-02-01')]

In [74]:
for index, row in ratings.iterrows():
    print(row)
    user_id = get_data_from_db(f"""
                SELECT user_id
                FROM users 
                WHERE nickname = ?
                """, (row.nickname,), out=True)
    print(user_id)
    boardgame_id = int(row.boardgame_id)
    rating = float(row.rating)
    num_of_plays = int(row.num_of_plays)
    comment_id = get_data_from_db(f"""
                SELECT comment_id
                FROM comments 
                WHERE comment = ?
                """, (row.comment,), out=True)
    print(comment_id)
    own = int(row.own)
    prevowned = int(row.prevowned)
    for_trade = int(row.for_trade)
    want = int(row.want)
    want_to_play = int(row.want_to_play)
    want_to_buy = int(row.want_to_buy)
    wishlist = int(row.wishlist)
    preordered = int(row.preordered)
    last_modified = row.last_modified[:10]
    print(last_modified)
    break

nickname                    DiVo1975
title                      Anachrony
boardgame_id                  185343
rating                           9.0
num_of_plays                       2
comment                         None
own                                0
prevowned                          1
fortrade                           0
want                               0
wanttoplay                         0
wanttobuy                          0
wishlist                           0
preordered                         0
last_modified    2022-11-06 14:02:30
Name: 0, dtype: object
[]
[]


AttributeError: 'Series' object has no attribute 'for_trade'

In [399]:
for index, row in boadrgames[boadrgames.year_published != ""].iterrows():
    if row.year_published != '0': # year_published
        year_published = int(row.year_published)
    else:
        year_published = np.nan
        
    if row.minplayers != '0': # min_players
        min_players = int(row.minplayers)
    else:
        min_players = np.nan
        
    if int(row.maxplayers) < min_players: # max_players
        max_players = np.nan
    elif row.maxplayers == '0':
        max_players = min_players
    else:
        max_players = int(row.maxplayers)
        
    if row.minplaytime != '0': # min_playtime
        min_playtime = int(row.minplaytime)
    else:
        min_playtime = np.nan
        
    if int(row.maxplaytime) < min_playtime: # max_playtime
        max_playtime = np.nan
    elif row.maxplaytime == '0':
        max_playtime = min_playtime
    else:
        max_playtime = int(row.maxplaytime)
        
    if (row.age != '0') & (row.age != ''): # age
        age = int(row.age)
    else:
        age = np.nan
        
    if row.users_rated != '': # users_rated
        users_rated = int(row.users_rated)
    else:
        users_rated = np.nan
        
    if row.average_rating != '': # average_rating
        average_rating = float(row.average_rating)
    else:
        average_rating = np.nan
        
    if row.bayes_average_rating != '': # bayes_average_rating
        bayes_average_rating = float(row.bayes_average_rating)
    else:
        bayes_average_rating = np.nan
    
    if row.stddev != '': # stddev
        std_dev = float(row.stddev)
    else:
        std_dev = np.nan
    
    if row.owned != '': # owned
        owned = int(row.owned)
    else:
        owned = np.nan 
        
    if row.trading != '': # trading
        trading = int(row.trading)
    else:
        trading = np.nan
    
    if row.wishing != '': # wishing
        wishing = int(row.wishing)
    else:
        wishing = np.nan 
        
    if row.num_of_comments != '': # num_of_comments
        num_of_comments = int(row.num_of_comments)
    else:
        num_of_comments = np.nan
        
    if row.num_of_weights != '': # num_of_weights
        num_of_weights = int(row.num_of_weights)
    else:
        num_of_weights = np.nan
        
    if row.average_weight != '': # average_weight
        average_weight = float(row.average_weight)
    else:
        average_weight = np.nan
        
    if (isNaN(row.main_publisher)) | (row.main_publisher == ''): # main_publisher_id
        main_publisher_id = np.nan
    else:
        main_publisher_id = get_data_from_db(f"""
                SELECT publisher_id
                FROM publishers 
                WHERE publisher = '{row.main_publisher}'
                """)[0][0]
        
    if (isNaN(row.thumbnail_link)) | (row.thumbnail_link == ''): # thumbnail_link_id
        thumbnail_link_id = np.nan
        
    else:
        thumbnail_link_id = get_data_from_db(f"""
                SELECT link_id
                FROM links 
                WHERE link = '{row.thumbnail_link}'
                """)[0][0]
        
    if (isNaN(row.image_link)) | (row.image_link == ''): # image_link_id
        image_link_id = np.nan
        
    else:
        image_link_id = get_data_from_db(f"""
                SELECT link_id
                FROM links 
                WHERE link = '{row.image_link}'
                """)[0][0]

    print(len((int(row.boardgame_id), row.title, year_published, min_players, max_players, 
           min_playtime, max_playtime, age, users_rated, average_rating, bayes_average_rating, std_dev,
           owned, trading, wishing, num_of_comments, 
           num_of_weights, average_weight, main_publisher_id, thumbnail_link_id, image_link_id)))
    break

        

21


In [392]:
get_data_from_db(f"""
                SELECT publisher_id
                FROM linkes 
                WHERE publisher = '{row.main_publisher}'
                """)[0][0]

5688

In [393]:
boadrgames[(boadrgames.year_published != "") & (boadrgames.thumbnail_link == "")].iloc[:,19:]



,ranks,main_publisher,description,publishers,honors,expansions,accessories,artists,mechanics,category,...,graphic_designers,subdomains,implementations,suggested_numplayers,podcast_episodes,comments,marketplace_history,marketplace_listings,thumbnail_link,image_link
17,Board Game Rank|Not Ranked||Abstract Game Rank...,(Self-Published),"In this card and dice-free horse racing game, ...",(Self-Published),,,,,Simulation,Abstract Strategy|Animals|Racing,...,,Abstract Games,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,Coudloc|Played once,,,,
24,Board Game Rank|Not Ranked,(Web published),"2 player traditional card game, combining tric...",(Web published),,,,,Melding and Splaying|Memory|Set Collection|Tri...,Card Game,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,,
54,Board Game Rank|Not Ranked,(Self-Published),"Overview<br/>Explore a game of Science, Techno...",(Self-Published),,,,,Moving Multiple Units|Pieces as Map|Real-Time|...,Civilization|Educational|Fantasy|Novel-based|R...,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,"bmcole|As the game designer, I find it only fi...",,,,
58,Board Game Rank|Not Ranked,Fitzpatrick Bros.,&quot;An exciting radio 'quiz' game for all th...,Fitzpatrick Bros.,,,,,,Card Game|Children's Game|Movies / TV / Radio ...,...,,,Meet the Missus,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,,
68,Board Game Rank|Not Ranked,D. Carvalho,"A race game based on Paul Pry (1825), a farcic...",D. Carvalho,,,,,Roll / Spin and Move,Dice,...,,,,0+||Best~0|Recommended~0|Not Recommended~0,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134640,Board Game Rank|Not Ranked,(Self-Published),Simple roll and move game based on characters ...,(Self-Published),,,,Ulf Grimås,Point to Point Movement|Roll / Spin and Move,Children's Game,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,,
134680,Board Game Rank|Not Ranked,(Self-Published),"Just like the name suggests, The Art of War pa...",(Self-Published),,,,,Area Movement|Deck Construction|Dice Rolling|G...,Civil War|Dice|Economic|Fantasy|Math|Medieval|...,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,,
134685,Board Game Rank|Not Ranked,(Self-Published),"Rock, Roadies and Roll is a game for 3 to 5 pl...",(Self-Published),,,,,Auction/Bidding|Worker Placement,Music,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,Man or Astroman|Summer 2012: First playtest co...,,,,
134697,Board Game Rank|Not Ranked,Fat Brain Toy Company,It&rsquo;s a silly sequencing memory game! Kid...,Fat Brain Toy Company,2017/Fall Parents' Choice Silver Honor Winner,,,,Memory,Animals|Card Game|Children's Game|Humor|Memory...,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,,


In [311]:
boadrgames.year_published.unique()

array(['2002', '2018', '2003', '2017', '2020', '2011', '2007', '1975',
       '2005', '2012', '2015', '1987', '2016', '2010', '1989', '2008',
       '2019', '2009', '2006', '0', '2021', '1991', '1976', '1979',
       '2022', '1995', '2014', '1992', '2023', '1940', '1998', '1977',
       '1811', '1984', '1988', '2013', '1983', '1999', '1970', '1967',
       '1985', '2001', '1969', '1993', '1937', '2004', '1996', '2000',
       '1990', '1986', '1994', '1809', '1962', '1973', '1981', '1953',
       '1972', '1997', '1978', '1897', '1951', '1980', '1947', '1964',
       '1959', '1961', '1925', '1700', '1915', '1960', '1954', '1974',
       '1982', '1966', '1888', '1930', '1936', '1968', '1926', '1945',
       '1903', '1921', '1971', '1900', '1938', '1957', '1932', '1935',
       '1950', '1928', '2024', '1810', '1955', '1890', '1956', '1958',
       '1949', '1963', '1863', '1701', '1948', '1870', '1965', '1913',
       '1791', '1898', '1931', '', '1908', '1896', '1923', '1922', '1910',
     

In [256]:
get_data_from_db("""
                SELECT city_id
                FROM cities 
                WHERE city = 'London'
                """)

[(6,)]

Order of adding info into database:
- country +
- state +
- city +
- users +

- links +
- publishers +
- boardgames +

- comments +
- ratings +

Other boardgame data
- descriptions
- boardgame_publishers
- honors
- boardgame_honors
- expansions
- boardgame_expansions
- accessories
- boardgame_accessories
- persons
- boardgame_artists
- mechanics
- boardgame_mechanics
- categories
- boardgame_categories
- boardgame_designers
- boardgame_graphic_designers
- subdomains
- implementations
- podcast_episodes
- boardgame_podcast_episodes
- sales
- boardgame_sales
- ranks

In [140]:
list(boardgames.columns)

['boardgame_id',
 'title',
 'year_published',
 'minplayers',
 'maxplayers',
 'minplaytime',
 'maxplaytime',
 'age',
 'users_rated',
 'average_rating',
 'bayes_average_rating',
 'median',
 'stddev',
 'owned',
 'trading',
 'wishing',
 'num_of_comments',
 'num_of_weights',
 'average_weight',
 'ranks',
 'main_publisher',
 'description',
 'publishers',
 'honors',
 'expansions',
 'accessories',
 'artists',
 'mechanics',
 'category',
 'designers',
 'graphic_designers',
 'subdomains',
 'implementations',
 'suggested_numplayers',
 'podcast_episodes',
 'comments',
 'marketplace_history',
 'marketplace_listings',
 'thumbnail_link',
 'image_link']

In [141]:
boardgames.honors[boardgames.honors != ""]

6         2011 Golden Geek Best 2-Player Board Game Nomi...
41        2018 Hungarian Board Game Award Nominee|2018 H...
99                        1987 Spiel des Jahres Recommended
122       2021 5 Seasons Best International Party&Coop N...
170             2011/Fall Parents' Choice Gold Award Winner
                                ...                        
134628     2020 UK Games Expo Best Miniatures Rules Nominee
134649    2019 5 Seasons Best Portuguese Expansion Nomin...
134669    2011/Spring Parents' Choice Approved Award Winner
134697        2017/Fall Parents' Choice Silver Honor Winner
134707    1987 Charles S. Roberts Best Pre-World War II ...
Name: honors, Length: 6041, dtype: object

In [138]:
boardgames[boardgames.subdomains != ''].iloc[:,27]

6         Cooperative Game|Deck Construction|Events|Hand...
7                                              Dice Rolling
15                                            Modular Board
17                                               Simulation
19                   Memory|Pattern Building|Tile Placement
                                ...                        
134712                                                     
134713    Hand Management|Highest-Lowest Scoring|Pattern...
134718    Network and Route Building|Simulation|Stock Ho...
134733                          Hand Management|Negotiation
134737                                        Modular Board
Name: mechanics, Length: 28288, dtype: object

In [96]:
set(boardgames.subdomains[boardgames.subdomains != ''])

{'Abstract Games',
 "Abstract Games|Children's Games",
 "Abstract Games|Children's Games|Wargames",
 'Abstract Games|Customizable Games',
 'Abstract Games|Family Games',
 'Abstract Games|Party Games',
 'Abstract Games|Strategy Games',
 'Abstract Games|Thematic Games',
 'Abstract Games|Wargames',
 "Children's Games",
 "Children's Games|Customizable Games",
 "Children's Games|Family Games",
 "Children's Games|Party Games",
 "Children's Games|Strategy Games",
 "Children's Games|Thematic Games",
 "Children's Games|Wargames",
 'Customizable Games',
 'Customizable Games|Family Games',
 'Customizable Games|Strategy Games',
 'Customizable Games|Thematic Games',
 'Customizable Games|Wargames',
 'Family Games',
 'Family Games|Party Games',
 'Family Games|Strategy Games',
 'Family Games|Strategy Games|Thematic Games',
 'Family Games|Thematic Games',
 'Family Games|Wargames',
 'Party Games',
 'Party Games|Strategy Games',
 'Party Games|Thematic Games',
 'Party Games|Wargames',
 'Strategy Games',
 

In [102]:
def ranks(list_ranks):
    ranks = []
    for i in list_ranks:
        for j in i.split('||'):
            ranks.append(j.split('|')[0])
    return list(set(ranks))

ranks(list(boardgames.ranks[boardgames.ranks != '']))

['Customizable Rank',
 'Atari ST Rank',
 'RPG Item Rank',
 'Arcade Rank',
 'Strategy Game Rank',
 'Board Game Rank',
 "Children's Game Rank",
 'Family Game Rank',
 'Abstract Game Rank',
 'Board Game Issue Rank',
 'Party Game Rank',
 'Thematic Rank',
 'Commodore 64 Rank',
 'Accessory Rank',
 'Amiga Rank',
 'War Game Rank',
 'Video Game Rank']

In [128]:
def categories(list_categories):
    categories = []
    for i in list_categories:
        categories.append(i.split('|')[0])
    return list(set(categories))

categories(list(boardgames.category[boardgames.category != '']))

['Game System',
 'American Revolutionary War',
 'Aviation / Flight',
 'Trains',
 'Real-time',
 'Abstract Strategy',
 'Industry / Manufacturing',
 'Bluffing',
 'Nautical',
 'Mafia',
 'Zombies',
 'Wargame',
 'Expansion for Base-game',
 'Mythology',
 'Print  Play',
 'Video Game Theme',
 'Medieval',
 'Age of Reason',
 'Miniatures',
 'American Indian Wars',
 'Party Game',
 'Trivia',
 'Political',
 'Modern Warfare',
 'Fantasy',
 'Ancient',
 'Farming',
 'Environmental',
 'Deduction',
 'Animals',
 'Comic Book / Strip',
 'Prehistoric',
 'Mature / Adult',
 'Racing',
 'Civil War',
 'Print & Play',
 'Card Game',
 'Exploration',
 'Medical',
 'Renaissance',
 'Dice',
 'Fighting',
 'Humor',
 'World War II',
 'Math',
 'Napoleonic',
 'Book',
 'Pirates',
 'Educational',
 'Music',
 'Novel-based',
 'Word Game',
 'Puzzle',
 'Maze',
 'Electronic',
 'Pike and Shot',
 'Arabian',
 'Memory',
 'Sports',
 'Post-Napoleonic',
 'World War I',
 'Number',
 'Space Exploration',
 'Horror',
 'City Building',
 'Territory B

In [97]:
categories(list(set(boardgames.subdomains[boardgames.subdomains != ''])))

['Wargames',
 'Abstract Games',
 'Party Games',
 "Children's Games",
 'Thematic Games',
 'Customizable Games',
 'Family Games',
 'Strategy Games']

In [99]:
len(categories(list(set(boardgames.graphic_designers[boardgames.graphic_designers != ''])))) 

1623

In [103]:
len(categories(list(set(boardgames.designers[boardgames.designers != '']))))

27351

In [142]:
len(categories(list(set(boardgames.honors[boardgames.honors != ''])))) 

2306

In [146]:
years = []
award = []
for i in categories(list(set(boardgames.honors[boardgames.honors != '']))):
    years.append(int(i[:4]))
    award.append(str(i[5:]))
#     print(int(i[:4]), '---', str(i[5:]))

2019 --- Gioco dell’Anno Nominee
2001 --- Essener Feder Best Written Rules Winner
1998 --- Spiel des Jahres Recommended
2001 --- Charles S. Roberts Best Wargame Graphics Nominee
2017 --- Guldbrikken Best Children's Game Nominee
2018 --- Origins Awards Best Board Game Nominee
2017 --- Spiel des Jahres Recommended
2002 --- Fairplay À la carte Runner-up
1997 --- Fairplay À la carte Runner-up
2015 --- Årets Spill Best Family Game Nominee
1986 --- Gra Roku Game of the Year Winner
2018 --- Graf Ludo Best Family Game Graphics Winner
2010 --- Diana Jones Award for Excellence in Gaming Nominee
1995 --- Årets Spil Best Children's Game Winner
2020 --- UK Games Expo Best Miniatures Rules Judges Award Winner
1979 --- Spiel des Jahres "Beautiful Game" Winner
2018 --- RPC Fantasy Award Board Game Winner
2007 --- Lucca Game Best Card Game
2013 --- Golden Geek Best Board Game Expansion Nominee
2021 --- UK Games Expo Best Variant or Expansion Judges Award Winner
2019 --- Board Game Quest Awards Best Fam

In [170]:
def history(x):
    for i in x.split('|||'):
        print(i)
        break
history(boadrgames.marketplace_history[boadrgames.marketplace_history != ''].iloc[2])

unix_saledate|1301347058||saletime|120837||price|25.95||currency|USD||condition|new


In [ ]:
boadrgames.marketplace_history[boadrgames.marketplace_history != ''].iloc[2]

In [112]:
len(set(ranks(list(boardgames.comments[boardgames.comments != '']))) - set(users.nickname))

34222

In [158]:
len(split_once(list(boardgames[boardgames.accessories != ''].accessories)))

6794

In [ ]:
# # users
# for ind in tqdm(range(len(users))):
#     username = users.iloc[ind]["nickname"]
#     country = users.iloc[ind]["country"]
#     state = users.iloc[ind]["state"]
#     city = users.iloc[ind]["city"]
#     link = users.iloc[ind]["link"]
#     query = (f"""
#     UPDATE users
#     SET WorkComment = '1'
#     WHERE ID='{id_excel}'
#    """)
# #ниже отправляю по одному в бд
#     conn.execute(query)
# conn.close()
# eng.dispose()